In [1]:
import os

from arcgis.features import GeoAccessor
import arcpy
import sys
sys.path.append('../../src')
from geoai_retail import enrich_local as enrich
from ba_data_paths import ba_data
from geoai_retail import utils

%load_ext autoreload
%autoreload 2

In [2]:
raw_gdb = '../../data/raw/raw.gdb'
origin_fc = os.path.join(raw_gdb, 'block_groups')
origin_id_fld = 'ID'
dest_fc = os.path.join(raw_gdb, 'coffee')
dest_id_fld = 'LOCNUM'
weighting_fc = os.path.join(raw_gdb, 'block_points')

enrich_template_fc = '../../src/tests/test_data.gdb/block_groups_enrich_template'
test_origin_fc = '../../src/tests/test_data.gdb/block_groups'
test_dest_fc = '../../src/tests/test_data.gdb/coffee_pdx'

interim_gdb = '../../data/interim/interim.gdb'

In [3]:
enrich_vars_df = ba_data.enrich_vars_dataframe
enrich_vars_df.head()

,name,alias,collection_name,enrich_str,enrich_field_name
0,AGE0_CY,2018 Population Age <1,1yearincrements,1yearincrements.AGE0_CY,F1yearincrements_AGE0_CY
1,AGE1_CY,2018 Population Age 1,1yearincrements,1yearincrements.AGE1_CY,F1yearincrements_AGE1_CY
2,AGE2_CY,2018 Population Age 2,1yearincrements,1yearincrements.AGE2_CY,F1yearincrements_AGE2_CY
3,AGE3_CY,2018 Population Age 3,1yearincrements,1yearincrements.AGE3_CY,F1yearincrements_AGE3_CY
4,AGE4_CY,2018 Population Age 4,1yearincrements,1yearincrements.AGE4_CY,F1yearincrements_AGE4_CY


In [4]:
id_fld = 'ID'

arcpy.env.workspace = interim_gdb
already_enriched = [v.split('_')[-1] for v in arcpy.ListFeatureClasses('block_groups_enriched_*')]
enrich_colls = [c for c in ba_data.enrich_vars_dataframe['collection_name'].unique() if c not in already_enriched]

enrich_fc_lst = []
fail_lst = []

for collection in enrich_colls:
    try:
        out_fc = os.path.join(interim_gdb, f'block_groups_enriched_{collection}')
        enrich.enrich_by_collection(origin_fc, collection, out_fc)
        enrich_fc_lst.append(out_fc)
        
        print(f'Success - {collection}')
        
    except Exception as e:
        fail_lst.append(collection)
        
        print(f'Fail - {collection}')
        print(e)

for idx, fc in enumerate(enrich_fc_lst):
    if idx == 0:
        enrich_df = GeoAccessor.from_featureclass(fc)
        enrich_df.drop('OBJECTID', axis=1, inplace=True)
    else:
        add_df = GeoAccessor.from_featureclass(fc)
        new_cols = add_df.columns.difference(enrich_df.columns)
        add_df.set_index(id_fld, drop=True, inplace=True)
        enrich_df = enrich_df.merge(add_df[new_cols], on=id_fld, right_index=True)
        
    print(f'{fc} {idx+1}/{len(enrich_fc_lst)} {len(enrich_df.columns)}')

drop_fld_lst = [f.name for f in arcpy.ListFields(origin_fc) if f.name != id_fld and f.name.upper() != 'SHAPE']
drop_fld_lst = [col for col in drop_fld_lst if col in enrich_df.columns]
drop_fld_lst.append('aggregationMethod')
enrich_df.drop(drop_fld_lst, axis=1, inplace=True)
    
enrich_df.set_index(id_fld, drop=True, inplace=True)

for fc in enrich_fc_lst:
    arcpy.management.Delete(fc)

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

2019-08-08 21:41:34,778 - DEBUG - Splitting the enrich task into 2 chunks.
2019-08-08 21:41:34,780 - DEBUG - Starting to enrich 1/2
2019-08-08 21:41:58,192 - DEBUG - Finished enriching 1/2
2019-08-08 21:41:58,193 - DEBUG - Starting to enrich 2/2
2019-08-08 21:42:13,271 - DEBUG - Finished enriching 2/2


Success - 1yearincrements


...

Success - women


2019-08-08 22:16:56,691 - DEBUG - Splitting the enrich task into 2 chunks.
2019-08-08 22:16:56,694 - DEBUG - Starting to enrich 1/2
2019-08-08 22:17:02,832 - DEBUG - Finished enriching 1/2
2019-08-08 22:17:02,836 - DEBUG - Starting to enrich 2/2
2019-08-08 22:17:08,031 - DEBUG - Finished enriching 2/2


Success - yearbuilt


2019-08-08 22:17:13,701 - DEBUG - Splitting the enrich task into 2 chunks.
2019-08-08 22:17:13,704 - DEBUG - Starting to enrich 1/2
2019-08-08 22:17:19,699 - DEBUG - Finished enriching 1/2
2019-08-08 22:17:19,703 - DEBUG - Starting to enrich 2/2
2019-08-08 22:17:25,144 - DEBUG - Finished enriching 2/2


Success - yearmovedin
../../data/interim/interim.gdb\block_groups_enriched_1yearincrements 1/92 580
../../data/interim/interim.gdb\block_groups_enriched_5yearincrements 2/92 768
../../data/interim/interim.gdb\block_groups_enriched_agebyracebysex 3/92 2272
../../data/interim/interim.gdb\block_groups_enriched_AtRisk 4/92 2285
../../data/interim/interim.gdb\block_groups_enriched_AutomobilesAutomotiveProducts 5/92 2411
../../data/interim/interim.gdb\block_groups_enriched_BabyProductsToysGames 6/92 2465
../../data/interim/interim.gdb\block_groups_enriched_businesses 7/92 2538
../../data/interim/interim.gdb\block_groups_enriched_CivicActivitiesPoliticalAffiliation 8/92 2569
../../data/interim/interim.gdb\block_groups_enriched_clothing 9/92 2643
../../data/interim/interim.gdb\block_groups_enriched_ClothingShoesAccessories 10/92 2748
../../data/interim/interim.gdb\block_groups_enriched_commute 11/92 2782
../../data/interim/interim.gdb\block_groups_enriched_DaytimePopulation 12/92 2786
../../da

In [6]:
enrich_df.to_csv(os.path.join('../../data/interim', 'block_group_demographics.csv'))